<a href="https://colab.research.google.com/github/Aswin2808/AI/blob/main/RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [69]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense

# Load your dataset (replace 'your_dataset.csv' with your actual CSV file)
# Example: Load a CSV dataset
# Replace this path with your actual CSV file path
file_path = '/content/drive/MyDrive/Microsoft_Stock.csv'
data = pd.read_csv(file_path)

# Assuming your dataset has columns 'Date' and 'Close'
# Adjust this based on your dataset structure
prices = data[['Date', 'Close']].copy()

# Convert Date column to datetime
prices['Date'] = pd.to_datetime(prices['Date'])

# Sorting data by date (ascending order)
prices = prices.sort_values(by='Date')

# Extract all but the last date for training
train_data = prices.head(len(prices) - 1)

# Scaling the 'Close' prices
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_prices = scaler.fit_transform(train_data[['Close']])

# Function to create dataset for RNN with single feature (Close price)
def create_dataset(data, time_steps=1):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:(i + time_steps), 0])
        y.append(data[i + time_steps, 0])  # Next 'Close' value
    return np.array(X), np.array(y)

# Setting time steps
time_steps = 10  # Adjust as needed
X, y = create_dataset(scaled_prices, time_steps)

# Reshape input to be [samples, time steps, features]
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

# Building the RNN model
model = Sequential()
model.add(SimpleRNN(units=50, input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(units=1))

# Compiling the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Training the model (you can adjust epochs and batch size as needed)
model.fit(X, y, epochs=100, batch_size=32)

# Predicting the last 'Close' value based on all preceding dates
last_sequence = scaled_prices[-time_steps:].reshape((1, time_steps, 1))
predicted_close = model.predict(last_sequence)[0][0]

# Getting the last 'Date' and 'Close' value from the dataset
last_date = prices.iloc[-1]['Date']
actual_last_close = prices.iloc[-1]['Close']

# Inverse transform the predicted close value to get the actual value
predicted_close = scaler.inverse_transform([[predicted_close]])[0][0]

print(f'Actual Last Close: {actual_last_close:.2f}')
print(f'Predicted Last Close for {last_date.date()}: {predicted_close:.2f}')


Epoch 1/100
47/47 [==============================] - 2s 7ms/step - loss: 0.0075
Epoch 2/100
47/47 [==============================] - 0s 7ms/step - loss: 2.6895e-04
Epoch 3/100
47/47 [==============================] - 0s 6ms/step - loss: 1.9203e-04
Epoch 4/100
47/47 [==============================] - 0s 8ms/step - loss: 1.8213e-04
Epoch 5/100
47/47 [==============================] - 0s 7ms/step - loss: 1.8444e-04
Epoch 6/100
47/47 [==============================] - 0s 6ms/step - loss: 1.8529e-04
Epoch 7/100
47/47 [==============================] - 0s 9ms/step - loss: 1.7849e-04
Epoch 8/100
47/47 [==============================] - 0s 10ms/step - loss: 1.7376e-04
Epoch 9/100
47/47 [==============================] - 0s 6ms/step - loss: 1.7340e-04
Epoch 10/100
47/47 [==============================] - 0s 7ms/step - loss: 1.7910e-04
Epoch 11/100
47/47 [==============================] - 1s 15ms/step - loss: 1.7065e-04
Epoch 12/100
47/47 [==============================] - 1s 12ms/step - loss: 1